In [9]:
import pandas as pd
import numpy as np

from skrub import TableVectorizer
import xgboost as xgb
from sklearn.pipeline import Pipeline

import holidays

from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import StandardScaler, LabelEncoder

import datetime


In [10]:
# Import the files
df_train = pd.read_parquet("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/data/train.parquet")
df_test = pd.read_parquet("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/data/final_test.parquet")

In [11]:
# Add jour ferie data
jour_feries = (
    pd.read_csv(
        "/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/external_data/jours_feries_metropole.csv",
        date_format="%Y%m%d%H"  # Ensure date format is handled correctly
    )
    .drop(columns=["annee", "zone"])  # Drop unnecessary columns
)

# Convert 'date' column to datetime
jour_feries['date'] = pd.to_datetime(jour_feries['date'])

# Filter rows based on the date range of df_train and df_test
jour_feries = jour_feries[
    (jour_feries["date"] >= df_train["date"].min() - datetime.timedelta(hours=1))
    & (jour_feries["date"] <= df_test["date"].max() + datetime.timedelta(hours=1))
]

In [12]:
# Extract the date feature on different time scales :

fr_holidays = holidays.France()

def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour

    # creation of a binary varible depicting if day in weekend
    X["is_weekend"] = np.where(X["weekday"] + 1 > 5, 1, 0)

    # Add a feature to indicate if the day is a holiday in France
    X["is_holiday"] = X["date"].apply(lambda d: 1 if d in fr_holidays else 0)

    # and if it is a jour ferie in France :
    X["is_jour_ferie"] = X["date"].dt.date.isin(jour_feries['date']).astype(int)

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

df_train = _encode_dates(df_train)
df_test = _encode_dates(df_test)


In [13]:
# Preprocessing :

# Drop unnecessary columns :
# df_train = df_train.drop(columns=["coordinates", "counter_name", "site_name", "counter_technical_id"])
# df_test = df_test.drop(columns=["coordinates", "counter_name", "site_name", "counter_technical_id"])

# Extract features from counter_installation_date
for df in [df_train, df_test]:
    df["installation_year"] = df["counter_installation_date"].dt.year
    df["installation_month"] = df["counter_installation_date"].dt.month

df_train = df_train.drop(columns=["counter_installation_date"])
df_test = df_test.drop(columns=["counter_installation_date"])

# Label encode high-cardinality categorical features
label_encoders = {}
for col in ["counter_id", "site_id", "counter_name", "site_name", "counter_technical_id", "coordinates"]:
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col])
    df_test[col] = le.fit_transform(df_test[col])
    label_encoders[col] = le

In [14]:
X_train = df_train.drop(columns=["bike_count", "log_bike_count"])
y_train = df_train["log_bike_count"]

X_test = df_test.copy()

In [15]:
# Define the XGBoost model
xgb_model = xgb.XGBRegressor()

# Define the parameter grid for tuning
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 6, 9, 12],
    'subsample': [0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.6, 0.8, 1.0],
}

In [16]:
# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    n_iter=50,  # Number of parameter settings sampled
    scoring='neg_mean_squared_error',  # Use appropriate scoring metric
    cv=5,  # 5-fold cross-validation
    verbose=1,
    random_state=42,
    n_jobs=-1  # Use all available processors
)

# Perform the search
random_search.fit(X_train, y_train)

# Get the best parameters and score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", -random_search.best_score_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Parameters: {'subsample': 0.8, 'n_estimators': 200, 'max_depth': 12, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
Best Score: 0.8886206685865161


In [17]:
# Update the model with the best parameters
best_xgb_model = random_search.best_estimator_

In [18]:
'''
# Preprocessing pipeline
pipeline = Pipeline(
    steps=[
        ('preprocessor', TableVectorizer()),
        ('model', best_xgb_model),
    ]
)
'''

"\n# Preprocessing pipeline\npipeline = Pipeline(\n    steps=[\n        ('preprocessor', TableVectorizer()),\n        ('model', best_xgb_model),\n    ]\n)\n"

In [19]:
# Initialize the XGBoost regressor
model = best_xgb_model

# Fit the model
model.fit(
    X_train, y_train,
)

# Make Predictions on Test Data
y_predictions = model.predict(X_test)

In [20]:
print(y_predictions)

[0.29689932 1.6576614  2.355469   ... 5.3124895  4.8178616  3.9409018 ]


In [21]:
pd.DataFrame(y_predictions, columns=["log_bike_count"]).reset_index().rename(
    columns={"index": "Id"}
).to_csv("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/predictions_option_2_vsimple.csv", index=False)